# Semantic Nyquist Prompt Ablation Test

We mimic prompt ablation by sampling different numbers of "clues" from a band-limited concept signal. Each concept is a combination of low-frequency Fourier modes (analogous to latent manifolds). We vary the number of observed tokens (samples) and measure reconstruction error. When the sample count drops below twice the concept complexity, reconstruction fails—mirroring the hypothesized semantic Nyquist threshold for LLM prompts.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
np.random.seed(0)



In [ ]:
def basis_matrix(t, complexity):
    cols = []
    for k in range(1, complexity + 1):
        cols.append(np.sin(2 * np.pi * k * t))
        cols.append(np.cos(2 * np.pi * k * t))
    return np.stack(cols, axis=1)


def simulate_error(complexity, sample_count, trials=200, noise=0.01):
    errors = []
    for _ in range(trials):
        coeffs = np.random.randn(2 * complexity)
        dense_t = np.linspace(0, 1, 200)
        dense_basis = basis_matrix(dense_t, complexity)
        concept = dense_basis @ coeffs

        sample_t = np.linspace(0, 1, sample_count)
        sample_basis = basis_matrix(sample_t, complexity)
        observations = sample_basis @ coeffs + noise * np.random.randn(sample_count)

        try:
            recon_coeffs, *_ = np.linalg.lstsq(sample_basis, observations, rcond=None)
            recon = dense_basis @ recon_coeffs
            mse = np.mean((concept - recon) ** 2)
        except np.linalg.LinAlgError:
            mse = np.inf
        errors.append(mse)
    return np.mean(errors)

complexities = [1, 2, 3, 4, 5]
sample_counts = np.arange(2, 18)
error_matrix = np.zeros((len(complexities), len(sample_counts)))

for i, comp in enumerate(complexities):
    for j, samples in enumerate(sample_counts):
        error_matrix[i, j] = simulate_error(comp, samples)



In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
mesh = ax.imshow(np.log10(error_matrix + 1e-8), aspect="auto", origin="lower",
                  extent=[sample_counts[0], sample_counts[-1], complexities[0], complexities[-1]], cmap="viridis")
ax.set_xlabel("Prompt samples (tokens)")
ax.set_ylabel("Concept complexity (Fourier modes)")
ax.set_title("Log10 reconstruction error")
ax.axline((0, 0), slope=0.5, color="white", linestyle="--", label="Nyquist boundary")
fig.colorbar(mesh, ax=ax, label="log10 MSE")
ax.legend(loc="upper right")
plt.show()



Error collapses once the sample count crosses roughly twice the concept complexity (white dashed line), matching the Nyquist condition $f_s \ge 2 f_{\max}$. Undersampled prompts (below the line) produce orders-of-magnitude larger reconstruction error, echoing the information-threshold hypothesis for LLM prompts.
